# KNIT7 : Enable Precision Timing Capability on Nodes
This notebook downloads the [FABRIC PTP Deployment repository](https://github.com/fabric-testbed/ptp) from github onto the nodes in your FABRIC slice. Ansible playbook in this repo would deploy the [linuxptp software package](https://linuxptp.sourceforge.net/) to enable PTP on your nodes. Sites used in the topology should be PTP capable.

## Import the FABlib and other Libraries

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import requests
import json

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice Name

In [ ]:
slice_name=f"Slice for KNIT7 Precision Timing Tutorial"
mySlice = fablib.get_slice(name = slice_name)

## Observe Slice Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Check if nodes in the slice are PTP Capable
* Not all FABRIC sites are PTP Capable. 
* The VMs use a virtual PTP device (ptp_kvm) which is time-synchronized by the host. 
* If the host is in sync with PTP, then the PTP time in the VMs should be accurate. 
* If the SITE/HOST is not PTP Capable, the VM may receive time in the virtual PTP device that may not be true PTP. 
* Even though the SITE/Node is not PTP Capable the software can still be installed and run but note that the timing observed will not be accurate.

In [ ]:
nodes = mySlice.get_nodes()
for node in nodes:
    print (f"{node.get_name()} is hosted on {node.get_host()}")
    ad = fablib.get_site_advertisement(node.get_site())
    print (f"PTP Capable: { ad.flags.ptp}\n")

## Install and setup linuxptp package  on nodes
Download the Ansible role to configure and install the LinuxPTP software. For more details regarding the steps performed in the playbook, please refer to the repo at [https://github.com/fabric-testbed/ptp](https://github.com/fabric-testbed/ptp)

In [ ]:
pre_requisites = None

# Set Deployment tool repository details
repo_branch = 'main'
repo_name = 'ptp'
destination_folder = f"""/tmp/{repo_name}-{repo_branch}"""
clone_instructions = f"""
cd /tmp/;rm -rf /tmp/{repo_name}-{repo_branch};git clone --branch {repo_branch} https://github.com/fabric-testbed/{repo_name}.git {destination_folder};
"""

### Setting PTP Install Restrictions

* If you do not want all interfaces synchronized to PTP, add the name of interfaces to avoid as shown
* Management interfaces are not considered and are avoided by default
* If you do not want the system clock synchronized to PTP set the 'SYNC_SYSTEM_CLOCK' to False
* If you do not have any restrictions for a node, you can omit that node from the list

Example:
```
NODE_RESTRICTIONS = { 
   'node1' : { 'AVOID_IFACES': ['enp6s0'],'SYNC_SYSTEM_CLOCK': False},
   'node2' : { 'AVOID_IFACES': ['enp6s0','enp7s0']},
}
```

In [ ]:
NODE_RESTRICTIONS = {}

### Restrict Ansible operation based on tags

* Possible values are ptp_stop,ptp_start,ptp_install 
* Only one tag is allowed
* If empty then all three are performed in the right sequence
* If NODE_RESTRICTIONS are applied along with the tags, the operations will not be performed on the AVOIDED INTERFACES

Example
```
ansible_tags = 'ptp_stop'
```

In [ ]:
ansible_tags = ''

### Run Ansible playbook on each node

In [ ]:
# Instruction to run ansible command from the node
ansible_instructions = f"""
cd {destination_folder}/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    node_name = node.get_name()
    
    # Create JSON files for extra params that will be provided to ansible
    if node_name in NODE_RESTRICTIONS.keys():    
        extra_ansible_params = f""" --extra-vars @parameters.json""";
        with open('/tmp/'+node_name+'-parameters.json', 'w') as f:
            json.dump(NODE_RESTRICTIONS[node_name], f)
        print (f"Uploading install restrictions for {node_name}")    
        node.upload_file('/tmp/'+node_name+'-parameters.json',destination_folder+'/ansible/parameters.json')
    else:
        extra_ansible_params = ''
    if ansible_tags != '':
        extra_ansible_params = extra_ansible_params + ' --tags '+ansible_tags
        
    print (f"Running the PTP Deployment Ansible Playbook on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}"\
                f"{extra_ansible_params}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()

print (f"Ansible Playbook run on all nodes completed\n")